# Preprocessing

In [2]:
!pip install librosa

In [3]:
!pip install audiolazy

     |████████████████████████████████| 122kB 21.6MB/s 


In [4]:
from audiolazy.lazy_lpc import lpc

In [5]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5887 sha256=b369d0bb7dee7bec7730eaa69ac03681ab84c76467fe69fbf014ce056e5f9200
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import scipy.io.wavfile as wavfile
import python_speech_features as pfs

In [7]:
def normalize_audio(audio):
    audio = audio / (np.abs(audio).max())
    return audio

In [8]:
def divide_audio(audio, resolution=100, window_duration=0.01, minimum_power=0.15, sample_rate=44100):    
    duration = len(audio) / sample_rate # in samples/sec
    iterations = int(duration * resolution)
    step = int(sample_rate / resolution)
    window_length = np.floor(sample_rate * window_duration)
    audio_power = np.square(normalize_audio(audio)) / window_length #Normalized power to window duration
    start = np.array([])
    stop = np.array([])
    is_started = False
    
    for n in range(iterations):
        power = np.sum(audio_power[n * step : int(n * step + window_length)]) # sensitive
        if not is_started and power > minimum_power:
            start = np.append(start, (n * step + window_length )/ 2 )
            is_started = True
        elif is_started and (power <= minimum_power or n == iterations-1):
            stop = np.append(stop, (n * step + window_length)/2 )
            is_started = False
    
    if start.size == 0:
        start = np.append(start, 0)
        stop = np.append(stop, len(audio))
        
    start = start.astype(int)
    stop = stop.astype(int) 
          
    return (start), (stop)

# MFCC

In [9]:
def get_mfcc(x, Fs):
  x = x.astype('float')
  m = librosa.feature.melspectrogram(x, Fs, hop_length= 4096)
  mfcc_coeff = librosa.feature.mfcc(x, Fs, S=m, n_mfcc=13) #np.asarray pfs.mfcc(x, Fs))
  #print("mfcc shape: {}".format(mfcc_coeff.shape))
  #print(mfcc_coeff.shape)
  #n = mfcc_coeff.shape[0]
  #mfcc_coeff = mfcc_coeff.reshape((n))
  d_mfcc = librosa.feature.delta(mfcc_coeff,width = 3,axis=0)
  dd_mfcc = librosa.feature.delta(d_mfcc,width = 3,axis=0)
  #mfcc_coeff = list(mfcc_coeff.reshape(1,13))
  return mfcc_coeff.tolist(), d_mfcc.tolist(), dd_mfcc.tolist()

# Formant

In [10]:
import sys
import wave
import math
from scipy.signal import lfilter, hamming
import librosa

"""
Estimate formants using LPC.
"""

def get_formants(x, Fs):

    # Get Hamming window.
    N = len(x)
    w = np.hamming(N)

    # Apply window and high pass filter.
    x1 = x * w
    x1 = lfilter([1], [1., 0.63], x1)

    ncoeff = int(2 + (Fs / 1000))
    #A = librosa.lpc(x1, ncoeff)
    A = lpc.autocor(x1, ncoeff)
    # Get roots.
    rts = np.roots(A.numlist)
    rts = [r for r in rts if np.imag(r) >= 0]

    # Get angles.
    angz = np.arctan2(np.imag(rts), np.real(rts))

    # Get frequencies
    frqs = sorted(angz * (Fs / (2 * math.pi)))

    return frqs


# Pitch

In [11]:
def get_pitch(x, Fs):
  fl = 512 if x.shape[0] < 2048 else 2048
  p = np.asarray(librosa.yin(x, 200, 350, Fs, frame_length=fl))
  if p.shape[0] >= 5:
    p = p[:5] 
  else:
    for i in range(5 - len(p)):
      p = np.append(p, 0)
  d_p = np.diff(p) 
  print("pitch shape:{}".format(p.shape))
  #except ParameterError:
   # d_p = p
  return p.tolist(), d_p.tolist()

# Jitter

In [12]:
def get_jitter(p):
  d = np.diff(p) if np.shape(p)[0] > 1 else p
  return (np.sum(np.absolute(d)) / (len(p) - 1))/ (np.sum(p)/len(p))

# Shimmer

In [13]:
def get_shimmer(x):
  d = np.diff(x) if x.shape[0] > 1 else x
  return (np.sum(np.absolute(np.diff(x))) / (len(x) - 1))/ (np.sum(x)/len(x))

# LPCC

In [14]:
def get_lpcc(x, Fs):
  ncoeff = int(2 + (Fs / 1000))
  return lpc.autocor(x, ncoeff).numlist

# Feature Extraction

In [140]:
import pandas as pd
'''mfcc = []
d_mfcc = []
dd_mfcc = []
formants = []
pitch = []
d_pitch = []
shimmer = []
jitter = []
lpcc = []'''
gender = []
data1 = []
data2 = []
data3 = []
data4 = []
data5 = []
data6 = []

In [16]:
from itertools import chain

In [139]:
def features(y, Fs, g):
  if g == 'M':
    gender.append(0)
  else:
    gender.append(1)
  a,b,c = get_mfcc(y, Fs)
  a = list(chain.from_iterable(a + b + c))
  d1 = list(map(float, a))
  data1.append(d1)
  #mfcc.append(get_mfcc(y, Fs))
  #d_mfcc.append(b)
  #dd_mfcc.append(c)
  #formants.append(get_formants(y, Fs))
  f = get_formants(y, Fs)
  p, d_p = get_pitch(y, Fs)
  #pitch.append(list(p))
  d2 = d1 + p
  data2.append(d2)
  d3 = d2 + d_p
  data3.append(d3)
  d4 = d3 + f
  data4.append(d4)
  #d_pitch.append(d_p)
  #shimmer.append(get_shimmer(y))
  #jitter.append(get_jitter(p))
  d5 = d4 + [get_shimmer(y), get_jitter(p)]
  data5.append(d5)
  #lpcc.append(get_lpcc(y, Fs)) 
  d6 = d5 + get_lpcc(y, Fs)
  data6.append(d6)

In [18]:
def feature_extract(x, Fs, g):
  start,end = divide_audio(x)
  for s,e in zip(start,end):
    y = x[s:e]
    #print(np.shape(y))
    while y.shape[0] > 2048:
      y1 = y[:2048]
      features(y1, Fs, g)
      y = y[2048:]
    features(y, Fs, g)

In [19]:
src_genders = ['01_M_native','02_M_nonNative','03_F_native', '04_M_nonNative', '05_F_native', '06_M_native', '07_F_native','08_F_nonNative', '09_F_nonNative', '10_M_nonNative','11_M_native']
src_mic = ['nao mic','port_mic','studio_mic']
src_folder = ['numbers','sentences']
src_numbers = ['one.wav','two.wav','three.wav','four.wav','five.wav','six.wav','seven.wav','eight.wav','nine.wav','ten.wav']
src_sentence = ['the_dog_is_in_front_of_the_horse.wav',
 'the_dog_is_on_top_of_the_shed.wav'
 'the_fish_is_in_the_pond.wav'
 'the_horse_is_behind_the_car.wav'
 'the_horse_is_next_to_the_stable.wav']

In [141]:
for g in src_genders:
  print(g)
  for mic in src_mic:
    for folder in src_folder:
      if folder == 'numbers':
        for number in src_numbers:
          try:
            Fs, x = wavfile.read('/content/drive/MyDrive/english_words_sentences/{}/{}/{}/{}'.format(g, mic,folder,number))
            x = x.astype('float')
            if x.ndim > 1:
              x = librosa.to_mono(x)
            print("x shape:{}".format(x.shape))
            feature_extract(x, Fs, g[3])
          except FileNotFoundError:
            continue
      else:
        for sentence in src_sentence:
          try:
            Fs, x = wavfile.read('/content/drive/MyDrive/english_words_sentences/{}/{}/{}/{}'.format(g, mic,folder,sentence))
            x = x.astype('float')
            if x.ndim > 1:
              x = librosa.to_mono(x)
            print("x shape:{}".format(x.shape))
            feature_extract(x, Fs, g[3])
          except FileNotFoundError:
            continue
      

01_M_native
x shape:(45303,)
pitch shape:(5,)


/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1985
  n_fft, y.shape[-1]


x shape:(44112,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)


/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1104
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=221
  n_fft, y.shape[-1]


pitch shape:(5,)
x shape:(58418,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(39342,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(40535,)
pitch shape:(5,)


...

pitch shape:(5,)
x shape:(43114,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(46088,)


/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=106
  n_fft, y.shape[-1]


pitch shape:(5,)
x shape:(37663,)
pitch shape:(5,)
x shape:(43610,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(41132,)
pitch shape:(5,)
pitch shape:(5,)
11_M_native
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(2,)
pitch shape:(5,)
x shape:(40634,)
pitch shape:(5,)


/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1039
  n_fft, y.shape[-1]


pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(34314,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(37925,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(39732,)
pitch shape:(5,)
x shape:(36119,)
pitch shape:(5,)
x shape:(40635,)
pitch shape:(5,)
x shape:(37022,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(33411,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
pitch shape:(5,)
x shape:(128224,)
pitch shape:(5,)


In [57]:
np.shape(data6)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(958,)

In [142]:
df1 = pd.DataFrame(data1)
#df1['Gender'] = gender
df2 = pd.DataFrame(data2)
#df2['Gender'] = gender
df3 = pd.DataFrame(data3)
#df3['Gender'] = gender
df4 = pd.DataFrame(data4)
#df4['Gender'] = gender
df5 = pd.DataFrame(data5)
#df5['Gender'] = gender
df6 = pd.DataFrame(data6)
#df6['Gender'] = gender[:-1]

In [143]:
df6.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
0,1.525869e+10,2.088751e+10,1.895929e+10,1.607517e+10,1.248387e+10,8.440750e+09,4.251744e+09,1.988300e+08,-3.510603e+09,-6.715604e+09,-9.280565e+09,-1.114596e+10,-1.234936e+10,1.850301e+09,1.850301e+09,-2.406169e+09,-3.237709e+09,-3.817210e+09,-4.116064e+09,-4.120960e+09,-3.881174e+09,-3.457217e+09,-2.884981e+09,-2.215176e+09,-1.534396e+09,-1.534396e+09,-2.128235e+09,-2.128235e+09,-2.544005e+09,-7.055205e+08,-4.391776e+08,-1.518751e+08,1.174454e+08,3.318715e+08,4.980964e+08,6.210204e+08,6.752922e+08,3.403900e+08,3.403900e+08,199.547511,...,0.015884,-0.068291,0.006674,0.008450,-0.023897,0.087046,-0.071542,0.037358,0.005823,-0.020620,0.023841,0.026660,-0.011535,-0.021308,0.007399,0.016737,-0.054681,0.016479,0.061512,-0.058561,0.020091,-0.023588,0.002215,0.054683,-0.033718,-0.020900,-0.013684,0.056745,-0.048060,0.029903,0.011481,-0.006057,-0.024990,0.009294,-0.013846,-0.006539,-0.006304,0.024064,NaN,NaN
1,1.019024e+10,1.369510e+10,1.206484e+10,1.040804e+10,9.140904e+09,8.224798e+09,7.502346e+09,6.855296e+09,6.224514e+09,5.555463e+09,4.800171e+09,3.971742e+09,3.169787e+09,9.373007e+08,9.373007e+08,-1.643526e+09,-1.461969e+09,-1.091623e+09,-8.192787e+08,-6.847507e+08,-6.389164e+08,-6.499168e+08,-7.121711e+08,-7.918605e+08,-8.151924e+08,-8.151924e+08,-1.290413e+09,-1.290413e+09,-1.199635e+09,2.759516e+08,3.213450e+08,2.034362e+08,9.018111e+07,1.741691e+07,-3.662731e+07,-7.097182e+07,-5.151067e+07,-1.166596e+07,-1.166596e+07,208.899812,...,-0.041773,0.022487,-0.020590,0.042284,0.012122,0.022545,-0.043149,-0.019938,0.011740,0.015319,0.008481,-0.032644,0.006058,0.012161,-0.037462,0.026300,0.005529,-0.006441,0.014307,0.013890,-0.006387,0.017281,-0.067646,0.013939,0.060146,-0.017774,-0.037005,0.011385,-0.002660,0.033343,-0.043996,0.027276,0.018788,-0.035792,-0.008022,-0.003613,0.011402,0.011133,NaN,NaN
2,1.955563e+10,2.680255e+10,2.482049e+10,2.261296e+10,2.046045e+10,1.815836e+10,1.553131e+10,1.267847e+10,9.838943e+09,7.208583e+09,4.907401e+09,2.973826e+09,1.385940e+09,2.632433e+09,2.632433e+09,-2.094796e+09,-2.180023e+09,-2.227301e+09,-2.464571e+09,-2.739944e+09,-2.846181e+09,-2.734943e+09,-2.465771e+09,-2.117379e+09,-1.760730e+09,-1.760730e+09,-2.363615e+09,-2.363615e+09,-2.406228e+09,-6.625268e+07,-1.422740e+08,-2.563215e+08,-1.908050e+08,2.500400e+06,1.902050e+08,3.087822e+08,3.525204e+08,1.783242e+08,1.783242e+08,199.547511,...,0.011902,0.019218,-0.034978,0.005468,0.064321,0.027944,-0.074303,-0.034617,0.083368,-0.000056,-0.040564,-0.000620,-0.018067,0.021119,0.029810,-0.048847,0.000018,0.056232,-0.020475,-0.048462,0.026011,-0.008905,0.041963,-0.008891,0.003998,-0.042344,0.027895,-0.009352,0.041799,-0.025819,-0.011736,0.035565,-0.035510,-0.031193,0.017015,0.031482,-0.005761,-0.005034,NaN,NaN
3,2.090971e+10,2.836255e+10,2.558187e+10,2.231779e+10,1.896864e+10,1.587559e+10,1.315706e+10,1.059768e+10,8.003988e+09,5.606886e+09,3.846558e+09,2.737570e+09,2.215563e+09,2.336079e+09,2.336079e+09,-3.022378e+09,-3.306612e+09,-3.221101e+09,-2.905792e+09,-2.638954e+09,-2.576535e+09,-2.495399e+09,-2.078715e+09,-1.434658e+09,-8.154973e+08,-8.154973e+08,-2.679228e+09,-2.679228e+09,-2.821345e+09,-9.936155e+07,2.004098e+08,2.910736e+08,1.646286e+08,7.177759e+07,2.489100e+08,5.303705e+08,6.316088e+08,3.095802e+08,3.095802e+08,218.560101,...,-0.132561,-0.010586,0.096485,-0.004893,0.002781,0.000602,-0.051876,0.100460,-0.006718,-0.037435,-0.039504,0.009255,0.015990,-0.013661,0.035309,-0.042386,-0.031103,0.099278,0.012274,-0.072419,-0.019959,0.055127,-0.016420,0.025027,-0.065618,0.025639,-0.000834,0.020546,0.009538,-0.046123,0.048214,0.009783,-0.010563,-0.019048,-0.006936,-0.001218,-0.017686,0.028474,NaN,NaN
4,2.363914e+10,3.266900e+10,3.091350e+10,2.907131e+10,2.750239e+10,2.597868e+10,2.416748e+10,2.19

In [144]:
import sklearn as sk

In [159]:
Y = np.asarray(list(map(int, gender)))
kf = sk.model_selection.StratifiedKFold(n_splits=5, shuffle = True)

In [156]:
def train_data(X, Y):
  for train_index, test_index in kf.split(X, Y):
    X_train, Y_train  = X.reindex(index = train_index), Y[train_index]
    X_test, Y_test = X.reindex(index = test_index), Y[test_index]    
  return X_train, Y_train, X_test, Y_test

In [162]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

accuracy_ann = []
def ann(X,Y):
  X_train, Y_train, X_test, Y_test = train_data(X,Y)
  n = X_train.shape[1]
  model = Sequential()
  model.add(Dense((n+1)/2, input_dim=n, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(x = X_train, y =Y_train, epochs=200, batch_size=10)
  y_pred = list(map(int,model.predict(X_test)))
  accuracy_ann.append(accuracy_score(y_pred,Y_test))

In [163]:
ann(df1, Y)

Epoch 1/200
77/77 [==============================] - 1s 1ms/step - loss: 8748576984.6154 - accuracy: 0.4653
Epoch 2/200
77/77 [==============================] - 0s 1ms/step - loss: 751094857.0256 - accuracy: 0.6479
Epoch 3/200
77/77 [==============================] - 0s 1ms/step - loss: 429615851.8974 - accuracy: 0.5276
Epoch 4/200
77/77 [==============================] - 0s 1ms/step - loss: 546396475.6923 - accuracy: 0.5881
Epoch 5/200
77/77 [==============================] - 0s 1ms/step - loss: 354175930.7692 - accuracy: 0.6023
Epoch 6/200
77/77 [==============================] - 0s 1ms/step - loss: 422796856.8408 - accuracy: 0.6019
Epoch 7/200
77/77 [==============================] - 0s 1ms/step - loss: 333638568.2051 - accuracy: 0.6056
Epoch 8/200
77/77 [==============================] - 0s 1ms/step - loss: 249504947.4615 - accuracy: 0.6133
Epoch 9/200
77/77 [==============================] - 0s 1ms/step - loss: 389330214.9744 - accuracy: 0.5704
Epoch 10/200
77/77 [================

In [164]:
ann(df2, Y)

Epoch 1/200
77/77 [==============================] - 1s 1ms/step - loss: 2947568072.2051 - accuracy: 0.4142
Epoch 2/200
77/77 [==============================] - 0s 1ms/step - loss: 461628736.4103 - accuracy: 0.5682
Epoch 3/200
77/77 [==============================] - 0s 1ms/step - loss: 437810113.6410 - accuracy: 0.5958
Epoch 4/200
77/77 [==============================] - 0s 1ms/step - loss: 307779601.6410 - accuracy: 0.5618
Epoch 5/200
77/77 [==============================] - 0s 1ms/step - loss: 384972298.0513 - accuracy: 0.5828
Epoch 6/200
77/77 [==============================] - 0s 1ms/step - loss: 397465935.7949 - accuracy: 0.5970
Epoch 7/200
77/77 [==============================] - 0s 1ms/step - loss: 228577618.1026 - accuracy: 0.6313
Epoch 8/200
77/77 [==============================] - 0s 1ms/step - loss: 197026758.9744 - accuracy: 0.5955
Epoch 9/200
77/77 [==============================] - 0s 1ms/step - loss: 245232931.6923 - accuracy: 0.5984
Epoch 10/200
77/77 [================

In [165]:
ann(df3, Y)

Epoch 1/200
77/77 [==============================] - 1s 1ms/step - loss: 2535850862.2564 - accuracy: 0.6138
Epoch 2/200
77/77 [==============================] - 0s 1ms/step - loss: 1043365579.4872 - accuracy: 0.5974
Epoch 3/200
77/77 [==============================] - 0s 1ms/step - loss: 409046414.0513 - accuracy: 0.5828
Epoch 4/200
77/77 [==============================] - 0s 1ms/step - loss: 330943559.1795 - accuracy: 0.5689
Epoch 5/200
77/77 [==============================] - 0s 1ms/step - loss: 571173611.8974 - accuracy: 0.5188
Epoch 6/200
77/77 [==============================] - 0s 1ms/step - loss: 623630393.0256 - accuracy: 0.5892
Epoch 7/200
77/77 [==============================] - 0s 1ms/step - loss: 349557811.6923 - accuracy: 0.5675
Epoch 8/200
77/77 [==============================] - 0s 1ms/step - loss: 324625220.9231 - accuracy: 0.5798
Epoch 9/200
77/77 [==============================] - 0s 1ms/step - loss: 332355086.3590 - accuracy: 0.6118
Epoch 10/200
77/77 [===============

In [166]:
df4.dropna(axis = 1, inplace=True)
df4.shape

(958, 71)

In [167]:
ann(df4, Y)

Epoch 1/200
77/77 [==============================] - 1s 1ms/step - loss: 3012877315.6410 - accuracy: 0.6478
Epoch 2/200
77/77 [==============================] - 0s 1ms/step - loss: 627045348.9231 - accuracy: 0.5887
Epoch 3/200
77/77 [==============================] - 0s 1ms/step - loss: 419944253.9487 - accuracy: 0.5913
Epoch 4/200
77/77 [==============================] - 0s 1ms/step - loss: 447523952.0000 - accuracy: 0.6226
Epoch 5/200
77/77 [==============================] - 0s 1ms/step - loss: 1104077430.9744 - accuracy: 0.5885
Epoch 6/200
77/77 [==============================] - 0s 1ms/step - loss: 411223914.6667 - accuracy: 0.5981
Epoch 7/200
77/77 [==============================] - 0s 1ms/step - loss: 424883310.3590 - accuracy: 0.6114
Epoch 8/200
77/77 [==============================] - 0s 1ms/step - loss: 560824121.5897 - accuracy: 0.5585
Epoch 9/200
77/77 [==============================] - 0s 1ms/step - loss: 185946529.7436 - accuracy: 0.6756
Epoch 10/200
77/77 [===============

In [168]:
df5.dropna(axis = 1, inplace=True)
df5.shape

(958, 73)

In [169]:
ann(df5, Y)

Epoch 1/200
77/77 [==============================] - 1s 1ms/step - loss: 1526141650.0513 - accuracy: 0.6184
Epoch 2/200
77/77 [==============================] - 0s 1ms/step - loss: 417688523.1795 - accuracy: 0.5807
Epoch 3/200
77/77 [==============================] - 0s 1ms/step - loss: 313754437.6410 - accuracy: 0.5790
Epoch 4/200
77/77 [==============================] - 0s 1ms/step - loss: 301421887.3846 - accuracy: 0.5742
Epoch 5/200
77/77 [==============================] - 0s 1ms/step - loss: 225610567.8205 - accuracy: 0.6227
Epoch 6/200
77/77 [==============================] - 0s 1ms/step - loss: 252667406.3590 - accuracy: 0.6007
Epoch 7/200
77/77 [==============================] - 0s 1ms/step - loss: 208284654.1282 - accuracy: 0.6220
Epoch 8/200
77/77 [==============================] - 0s 1ms/step - loss: 385056041.1795 - accuracy: 0.5630
Epoch 9/200
77/77 [==============================] - 0s 1ms/step - loss: 233366120.7143 - accuracy: 0.6176
Epoch 10/200
77/77 [================

In [170]:
df6.dropna(axis = 1, inplace=True)
df6.shape

(958, 120)

In [172]:
ann(df6, Y)

Epoch 1/200
77/77 [==============================] - 1s 1ms/step - loss: 3712869727.1795 - accuracy: 0.4821
Epoch 2/200
77/77 [==============================] - 0s 1ms/step - loss: 578223617.0256 - accuracy: 0.5727
Epoch 3/200
77/77 [==============================] - 0s 1ms/step - loss: 780705319.5897 - accuracy: 0.5988
Epoch 4/200
77/77 [==============================] - 0s 1ms/step - loss: 369461856.0000 - accuracy: 0.5551
Epoch 5/200
77/77 [==============================] - 0s 1ms/step - loss: 305831816.4103 - accuracy: 0.6100
Epoch 6/200
77/77 [==============================] - 0s 1ms/step - loss: 355309289.4359 - accuracy: 0.5844
Epoch 7/200
77/77 [==============================] - 0s 1ms/step - loss: 220674301.9487 - accuracy: 0.6419
Epoch 8/200
77/77 [==============================] - 0s 1ms/step - loss: 175819461.0769 - accuracy: 0.6046
Epoch 9/200
77/77 [==============================] - 0s 1ms/step - loss: 283861794.6667 - accuracy: 0.6125
Epoch 10/200
77/77 [================

In [173]:
for i in range(len(accuracy_ann)):
  print('Accuracy is for the feature combination {}: {}'.format(i,accuracy_ann[i]*100))

Accuracy is for the feature combination 0: 52.35602094240838
Accuracy is for the feature combination 1: 43.455497382198956
Accuracy is for the feature combination 2: 52.35602094240838
Accuracy is for the feature combination 3: 59.16230366492147
Accuracy is for the feature combination 4: 65.96858638743456
Accuracy is for the feature combination 5: 51.30890052356021


In [208]:
np.mean(accuracy_ann)*100

54.10122164048865

# DNN

In [174]:
accuracy_dnn = []
def dnn(X,Y):
  X_train, Y_train, X_test, Y_test = train_data(X,Y)
  n = X_train.shape[1]
  dnn_model = Sequential()
  dnn_model.add(Dense(n, input_dim=n, activation='relu'))
  dnn_model.add(Dense(n, activation='relu'))
  dnn_model.add(Dense(n, activation='relu'))
  dnn_model.add(Dense(1, activation='sigmoid'))
  dnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  dnn_model.fit(x = X_train, y =Y_train, epochs=200, batch_size=30)
  y_pred = list(map(int,dnn_model.predict(X_test)))
  #y_pred = y_pred.astype(dtype='int')
  accuracy_dnn.append(accuracy_score(y_pred,Y_test))

In [175]:
dnn(df1,Y)

Epoch 1/200
26/26 [==============================] - 1s 2ms/step - loss: 2574235282.9630 - accuracy: 0.5169
Epoch 2/200
26/26 [==============================] - 0s 1ms/step - loss: 616277984.0000 - accuracy: 0.5319
Epoch 3/200
26/26 [==============================] - 0s 1ms/step - loss: 176820810.6667 - accuracy: 0.5393
Epoch 4/200
26/26 [==============================] - 0s 1ms/step - loss: 230322226.9630 - accuracy: 0.5717
Epoch 5/200
26/26 [==============================] - 0s 1ms/step - loss: 284780158.8148 - accuracy: 0.6302
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 165553173.6296 - accuracy: 0.6048
Epoch 7/200
26/26 [==============================] - 0s 1ms/step - loss: 415234420.1481 - accuracy: 0.5538
Epoch 8/200
26/26 [==============================] - 0s 1ms/step - loss: 241762628.2963 - accuracy: 0.5650
Epoch 9/200
26/26 [==============================] - 0s 1ms/step - loss: 143028666.6667 - accuracy: 0.6061
Epoch 10/200
26/26 [================

In [176]:
dnn(df2,Y)

Epoch 1/200
26/26 [==============================] - 1s 2ms/step - loss: 2108677978.0741 - accuracy: 0.4610
Epoch 2/200
26/26 [==============================] - 0s 1ms/step - loss: 309766323.5556 - accuracy: 0.6526
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 350919457.1852 - accuracy: 0.5548
Epoch 4/200
26/26 [==============================] - 0s 1ms/step - loss: 160307533.5556 - accuracy: 0.6278
Epoch 5/200
26/26 [==============================] - 0s 1ms/step - loss: 113868289.1852 - accuracy: 0.6375
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 111699188.2963 - accuracy: 0.6025
Epoch 7/200
26/26 [==============================] - 0s 1ms/step - loss: 255117178.0741 - accuracy: 0.6514
Epoch 8/200
26/26 [==============================] - 0s 1ms/step - loss: 76227624.5926 - accuracy: 0.6054
Epoch 9/200
26/26 [==============================] - 0s 2ms/step - loss: 48651918.3704 - accuracy: 0.6441
Epoch 10/200
26/26 [==================

In [177]:
dnn(df3,Y)

Epoch 1/200
26/26 [==============================] - 1s 1ms/step - loss: 486710533.9259 - accuracy: 0.5998
Epoch 2/200
26/26 [==============================] - 0s 1ms/step - loss: 189472375.1111 - accuracy: 0.5614
Epoch 3/200
26/26 [==============================] - 0s 1ms/step - loss: 127485962.9630 - accuracy: 0.6396
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 198269551.4074 - accuracy: 0.5871
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 208602897.7778 - accuracy: 0.6125
Epoch 6/200
26/26 [==============================] - 0s 1ms/step - loss: 140600884.1481 - accuracy: 0.6053
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 115726008.8889 - accuracy: 0.6930
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 98001677.2593 - accuracy: 0.5659
Epoch 9/200
26/26 [==============================] - 0s 2ms/step - loss: 81458464.2222 - accuracy: 0.5766
Epoch 10/200
26/26 [===================

In [178]:
dnn(df4,Y)

Epoch 1/200
26/26 [==============================] - 1s 2ms/step - loss: 1186109736.2963 - accuracy: 0.6101
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 314729253.9259 - accuracy: 0.6634
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 175566091.2593 - accuracy: 0.5981
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 115098245.0370 - accuracy: 0.6680
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 249188890.0741 - accuracy: 0.6323
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 251800209.7407 - accuracy: 0.6644
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 206815912.2963 - accuracy: 0.6010
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 182179898.0741 - accuracy: 0.5631
Epoch 9/200
26/26 [==============================] - 0s 2ms/step - loss: 257760832.8889 - accuracy: 0.5999
Epoch 10/200
26/26 [================

In [179]:
dnn(df5,Y)

Epoch 1/200
26/26 [==============================] - 1s 2ms/step - loss: 834240128.0000 - accuracy: 0.5618
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 307493540.1481 - accuracy: 0.5728
Epoch 3/200
26/26 [==============================] - 0s 1ms/step - loss: 196827102.5185 - accuracy: 0.6194
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 248267334.2222 - accuracy: 0.5350
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 171947856.5926 - accuracy: 0.5999
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 130735615.4074 - accuracy: 0.6226
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 65613836.4444 - accuracy: 0.6233
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 170055180.4444 - accuracy: 0.6485
Epoch 9/200
26/26 [==============================] - 0s 2ms/step - loss: 120577857.0370 - accuracy: 0.6108
Epoch 10/200
26/26 [==================

In [181]:
dnn(df6,Y)

Epoch 1/200
26/26 [==============================] - 1s 2ms/step - loss: 1056941696.0000 - accuracy: 0.6059
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 230936832.7407 - accuracy: 0.6168
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 363644206.2222 - accuracy: 0.5644
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 205507264.1481 - accuracy: 0.6658
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 168227598.8148 - accuracy: 0.5435
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 145076076.4444 - accuracy: 0.6114
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 128682141.3333 - accuracy: 0.6151
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 241599310.8148 - accuracy: 0.5970
Epoch 9/200
26/26 [==============================] - 0s 2ms/step - loss: 99977008.8148 - accuracy: 0.6218
Epoch 10/200
26/26 [=================

In [183]:
for i in range(len(accuracy_dnn)):
  print('Accuracy is for the feature combination {}: {}'.format(i,accuracy_dnn[i]*100)) 

Accuracy is for the feature combination 0: 58.63874345549738
Accuracy is for the feature combination 1: 67.01570680628272
Accuracy is for the feature combination 2: 68.58638743455498
Accuracy is for the feature combination 3: 67.01570680628272
Accuracy is for the feature combination 4: 65.44502617801047
Accuracy is for the feature combination 5: 61.78010471204188


In [207]:
np.mean(accuracy_dnn)*100

64.74694589877835

# Random Forest

In [188]:
from sklearn.ensemble import RandomForestClassifier
accuracy_rf = []

def rf(X, Y):
  X_train, Y_train, X_test, Y_test = train_data(X,Y)
  clf = RandomForestClassifier(n_estimators = 100)
  clf.fit(X_train, Y_train)
  y_pred = list(map(int,clf.predict(X_test)))
  accuracy_rf.append(accuracy_score(y_pred,Y_test))

In [189]:
rf(df1, Y)

In [190]:
rf(df2, Y)

In [191]:
rf(df3, Y)

In [192]:
rf(df4, Y)

In [193]:
rf(df5, Y)

In [194]:
rf(df6, Y)

In [195]:
for i in range(len(accuracy_rf)):
  print('Accuracy is for the feature combination {}: {}'.format(i,accuracy_rf[i]*100)) 

Accuracy is for the feature combination 0: 69.10994764397905
Accuracy is for the feature combination 1: 70.68062827225131
Accuracy is for the feature combination 2: 70.68062827225131
Accuracy is for the feature combination 3: 74.3455497382199
Accuracy is for the feature combination 4: 74.3455497382199
Accuracy is for the feature combination 5: 70.68062827225131


In [206]:
np.mean(accuracy_rf)*100

71.64048865619547